In [1]:
 !pip install -q tensorflow-model-optimization

In [2]:
from tensorflow import keras
import sys
import os
import h5py
import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
relative_fold = '/content/drive/MyDrive/bad_net'

def join_name(name):
    return os.path.join(relative_fold, name)

In [5]:
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))
    
    return x_data, y_data

In [6]:
def eval(x_test, y_test, bd_model):
    label_p = np.argmax(bd_model.predict(x_test), axis=1)
    accuracy = np.mean(np.equal(label_p, y_test))*100

    return accuracy

In [7]:
# Singal target
clean_validation_data_path = join_name("data/clean_validation_data.h5")
clean_test_data_path = join_name("data/clean_test_data.h5")
sunglass_poisoned_data_path = join_name("data/sunglasses_poisoned_data.h5")
anonymous_poisoned_data_path = join_name("data/anonymous_1_poisoned_data.h5")

# Multi-trigger Multi-target
eyebrows_poisoned_multi_data_path = join_name("data/Multi-trigger Multi-target/eyebrows_poisoned_data.h5")
lipstick_poisoned_multi_data_path = join_name("data/Multi-trigger Multi-target/lipstick_poisoned_data.h5")
sunglass_poisoned_multi_data_path = join_name("data/Multi-trigger Multi-target/sunglasses_poisoned_data.h5")

In [8]:
clean_validation_x, clean_validation_y = data_loader(clean_validation_data_path)
clean_test_x, clean_test_y = data_loader(clean_test_data_path)
sunglass_poisoned_test_x, sunglass_poisoned_t_y = data_loader(sunglass_poisoned_data_path)
anonymous_poisoned_x, anonymous_poisoned_y = data_loader(anonymous_poisoned_data_path)

# multi_trigger_multi_target
eyebrows_poisoned_x, eyebrows_poisoned_y = data_loader(eyebrows_poisoned_multi_data_path)
lipstick_poisoned_x, lipstick_poisoned_y = data_loader(lipstick_poisoned_multi_data_path)
sunglass_poisoned_x, sunglass_poisoned_y = data_loader(sunglass_poisoned_multi_data_path)

bd1: sunglasses_bd_net \\
bd2: anonymous_1_bd_net \\
bd3: anonymous_2_bd_net \\
bd4: multi_trigger_multi_target_bd_net \\

In [9]:
bd1_path = join_name("models/sunglasses_bd_net.h5")
bd1_weights = join_name("models/sunglasses_bd_weights.h5")
bd1 = keras.models.load_model(bd1_path)
bd1.load_weights(bd1_weights)

bd2_path = join_name("models/anonymous_1_bd_net.h5")
bd2_weights = join_name("models/anonymous_1_bd_weights.h5")
bd2 = keras.models.load_model(bd2_path)
bd2.load_weights(bd2_weights)

bd3_path = join_name("models/anonymous_2_bd_net.h5")
bd3_weights = join_name("models/anonymous_2_bd_weights.h5")
bd3 = keras.models.load_model(bd3_path)
bd3.load_weights(bd3_weights)

# multi_trigger_multi_target
bd4_path = join_name("models/multi_trigger_multi_target_bd_net.h5")
bd4_weights = join_name("models/multi_trigger_multi_target_bd_weights.h5")
bd4 = keras.models.load_model(bd4_path)
bd4.load_weights(bd4_weights)

## Simple Prune method
https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras

In [10]:

import tempfile
def Prune(bd_model, train_data, train_labels): 
    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    # Compute end step to finish pruning after 2 epochs.
    # batch_size = 128
    epochs = 5
    validation_split = 0.1 # 10% of training set will be used for validation set.
    num_data = train_data.shape[0] * (1 - validation_split)
    end_step = np.ceil(num_data).astype(np.int32) * epochs 
    train_labels = keras.utils.to_categorical(train_labels, 1283)

    # Define model for pruning.
    pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
    }

    model_for_pruning = prune_low_magnitude(bd_model, **pruning_params)

    # `prune_low_magnitude` requires a recompile.
    # learning_rate=0.001, beta_1=0.9, beta_2=0.999
    model_for_pruning.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
    
    logdir = tempfile.mkdtemp()

    callbacks = [
      tfmot.sparsity.keras.UpdatePruningStep(),
      tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
    ]
    model_for_pruning.fit(train_data, train_labels,
                  epochs=epochs, validation_split=validation_split, 
                  callbacks=callbacks)

    return model_for_pruning

In [11]:
def single_pretty_print(bd):
    cl = eval(clean_test_x, clean_test_y, bd)
    sp = eval(sunglass_poisoned_test_x, sunglass_poisoned_t_y, bd)
    ap = eval(anonymous_poisoned_x, anonymous_poisoned_y, bd)
    return [cl, sp, ap]

def multiple_pretty_print(bd):
    ep = eval(eyebrows_poisoned_x, eyebrows_poisoned_y, bd)
    lp = eval(lipstick_poisoned_x, lipstick_poisoned_y, bd)
    sp = eval(sunglass_poisoned_x, sunglass_poisoned_y, bd)
    return [ep, lp, sp]

In [12]:
old_ac = {}
new_ac = {}

In [13]:
old_ac['sunglasses_bd_net'] = single_pretty_print(bd1)
new_bd1 = Prune(bd1, clean_validation_x, clean_validation_y)
new_ac['sunglasses_bd_net'] = single_pretty_print(new_bd1)

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)
/usr/local/lib/python3.7/dist-packages/keras/layers/wrappers.py:64: CustomMaskWarning: Custom mask layers require a config and mus

Epoch 1/5
325/325 [==============================] - 45s 123ms/step - loss: 151.1822 - accuracy: 0.6732 - val_loss: 30.2803 - val_accuracy: 0.7411
Epoch 2/5
325/325 [==============================] - 37s 114ms/step - loss: 10.0607 - accuracy: 0.8619 - val_loss: 15.7408 - val_accuracy: 0.7974
Epoch 3/5
325/325 [==============================] - 37s 113ms/step - loss: 2.7935 - accuracy: 0.9314 - val_loss: 11.9028 - val_accuracy: 0.8251
Epoch 4/5
325/325 [==============================] - 38s 117ms/step - loss: 1.2414 - accuracy: 0.9551 - val_loss: 11.5933 - val_accuracy: 0.8355
Epoch 5/5
325/325 [==============================] - 36s 112ms/step - loss: 0.7487 - accuracy: 0.9694 - val_loss: 9.9878 - val_accuracy: 0.8139


In [14]:
old_ac['anonymous_1_bd_net'] = single_pretty_print(bd2)
new_bd2 = Prune(bd2, clean_validation_x, clean_validation_y)
new_ac['anonymous_1_bd_net'] = single_pretty_print(new_bd2)

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)
/usr/local/lib/python3.7/dist-packages/keras/layers/wrappers.py:64: CustomMaskWarning: Custom mask layers require a config and mus

Epoch 1/5
325/325 [==============================] - 39s 107ms/step - loss: 100.6869 - accuracy: 0.7727 - val_loss: 29.6734 - val_accuracy: 0.8364
Epoch 2/5
325/325 [==============================] - 34s 105ms/step - loss: 12.1320 - accuracy: 0.9015 - val_loss: 19.9023 - val_accuracy: 0.8580
Epoch 3/5
325/325 [==============================] - 34s 106ms/step - loss: 4.7695 - accuracy: 0.9430 - val_loss: 16.1699 - val_accuracy: 0.8684
Epoch 4/5
325/325 [==============================] - 35s 107ms/step - loss: 2.5687 - accuracy: 0.9576 - val_loss: 14.9733 - val_accuracy: 0.8710
Epoch 5/5
325/325 [==============================] - 34s 106ms/step - loss: 2.0755 - accuracy: 0.9621 - val_loss: 14.0160 - val_accuracy: 0.8762


In [15]:
old_ac['anonymous_2_bd_net'] = single_pretty_print(bd3)
new_bd3 = Prune(bd3, clean_validation_x, clean_validation_y)
new_ac['anonymous_2_bd_net'] = single_pretty_print(new_bd3)

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)
/usr/local/lib/python3.7/dist-packages/keras/layers/wrappers.py:64: CustomMaskWarning: Custom mask layers require a config and mus

Epoch 1/5
325/325 [==============================] - 42s 116ms/step - loss: 103.9026 - accuracy: 0.7862 - val_loss: 47.4819 - val_accuracy: 0.8121
Epoch 2/5
325/325 [==============================] - 36s 111ms/step - loss: 16.5868 - accuracy: 0.9054 - val_loss: 29.2995 - val_accuracy: 0.8658
Epoch 3/5
325/325 [==============================] - 38s 117ms/step - loss: 6.3050 - accuracy: 0.9471 - val_loss: 25.5678 - val_accuracy: 0.8736
Epoch 4/5
325/325 [==============================] - 40s 122ms/step - loss: 5.0167 - accuracy: 0.9544 - val_loss: 25.0042 - val_accuracy: 0.8762
Epoch 5/5
325/325 [==============================] - 36s 110ms/step - loss: 3.5198 - accuracy: 0.9622 - val_loss: 20.6087 - val_accuracy: 0.8823


In [16]:
old_ac['multi_trigger_multi_target_bd_net'] = single_pretty_print(bd4)
new_bd4 = Prune(bd4, clean_validation_x, clean_validation_y)
new_ac['multi_trigger_multi_target_bd_net'] = single_pretty_print(new_bd4)

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)
/usr/local/lib/python3.7/dist-packages/keras/layers/wrappers.py:64: CustomMaskWarning: Custom mask layers require a config and mus

Epoch 1/5
325/325 [==============================] - 39s 106ms/step - loss: 102.1781 - accuracy: 0.7392 - val_loss: 35.5632 - val_accuracy: 0.8087
Epoch 2/5
325/325 [==============================] - 33s 103ms/step - loss: 12.9265 - accuracy: 0.8933 - val_loss: 25.7805 - val_accuracy: 0.8190
Epoch 3/5
325/325 [==============================] - 33s 102ms/step - loss: 5.1519 - accuracy: 0.9360 - val_loss: 22.9363 - val_accuracy: 0.8485
Epoch 4/5
325/325 [==============================] - 34s 105ms/step - loss: 3.3958 - accuracy: 0.9506 - val_loss: 21.7985 - val_accuracy: 0.8537
Epoch 5/5
325/325 [==============================] - 34s 104ms/step - loss: 3.1216 - accuracy: 0.9519 - val_loss: 20.5567 - val_accuracy: 0.8459


In [17]:
for k,v in old_ac.items():
    vn = new_ac[k]
    if k == 'multi_trigger_multi_target_bd_net':
        print(f'Before training {k} ......')
        print(f'Eyebrows AC = {v[0]:.2f}%')
        print(f'Lipstick AC = {v[1]:.2f}%')
        print(f'Sunglass AC = {v[2]:.2f}%')

        print(f'After training {k} ......')
        print(f'Eyebrows AC = {vn[0]:.2f}%')
        print(f'Lipstick AC = {vn[1]:.2f}%')
        print(f'Sunglass AC = {vn[2]:.2f}%')
    else:
        print(f'Before training {k} ......')
        print(f'Clean test AC = {v[0]:.2f}%')
        print(f'Sunglass AC = {v[1]:.2f}%')
        print(f'Anonymous 1 AC = {v[2]:.2f}%')

        print(f'After training {k} ......')
        print(f'Clean test AC = {vn[0]:.2f}%')
        print(f'Sunglass AC = {vn[1]:.2f}%')
        print(f'Anonymous 1 AC = {vn[2]:.2f}%')


Before training sunglasses_bd_net ......
Clean test AC = 42.36%
Sunglass AC = 99.95%
Anonymous 1 AC = 1.30%
After training sunglasses_bd_net ......
Clean test AC = 84.59%
Sunglass AC = 49.63%
Anonymous 1 AC = 0.16%
Before training anonymous_1_bd_net ......
Clean test AC = 56.23%
Sunglass AC = 0.00%
Anonymous 1 AC = 91.48%
After training anonymous_1_bd_net ......
Clean test AC = 87.14%
Sunglass AC = 0.00%
Anonymous 1 AC = 82.00%
Before training anonymous_2_bd_net ......
Clean test AC = 67.63%
Sunglass AC = 0.00%
Anonymous 1 AC = 0.00%
After training anonymous_2_bd_net ......
Clean test AC = 88.82%
Sunglass AC = 0.00%
Anonymous 1 AC = 0.00%
Before training multi_trigger_multi_target_bd_net ......
Eyebrows AC = 43.20%
Lipstick AC = 0.00%
Sunglass AC = 0.00%
After training multi_trigger_multi_target_bd_net ......
Eyebrows AC = 86.89%
Lipstick AC = 0.00%
Sunglass AC = 0.01%


## Save Model

In [18]:
import os

generate_folder = "new_model"
if not os.path.exists(generate_folder):
    os.mkdir(generate_folder)

new_bd1.save('new_model/sunglasses_bd_net.h5')
new_bd2.save('new_model/anonymous_1_bd_net.h5')
new_bd3.save('new_model/anonymous_2_bd_net.h5')
new_bd4.save('new_model/multi_trigger_multi_target_bd_net.h5')

/usr/local/lib/python3.7/dist-packages/keras/layers/wrappers.py:64: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  config = {'layer': generic_utils.serialize_keras_object(self.layer)}


## Data Augment + Prune

In [19]:
generate_folder = "new_model_data_aug"
if not os.path.exists(generate_folder):
    os.mkdir(generate_folder)

In [20]:
def data_agument(X, Y):
    a_log = np.zeros((X.shape))
    a_exp = np.zeros((X.shape))
    a_noise = np.zeros((X.shape))

    for ind in range(X.shape[0]):
        a_log[ind] = (np.log2(X[ind])/20)*255 # reduces contrast
        a_exp[ind] = (np.exp(X[ind]/32)) # increases contrast
        no = np.random.normal(0, 0.05, size=X[ind].shape)
        a_noise[ind] = X[ind] + no

    X_aug = np.concatenate((X, a_log, a_exp, a_noise))
    Y_aug = np.concatenate((Y, Y, Y, Y))

    return X_aug, Y_aug

In [21]:
clean_validation_x, clean_validation_y = data_agument(clean_validation_x, clean_validation_y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log2
  import sys


In [22]:
# old_ac is the same as previous experiment.
# old_ac = {}
new_ac = {}

In [23]:
bd = keras.models.load_model(bd1_path)
bd.load_weights(bd1_weights)

In [24]:
# old_ac['sunglasses_bd_net'] = single_pretty_print(bd1)
new_bd = Prune(bd, clean_validation_x, clean_validation_y)
new_ac['sunglasses_bd_net'] = single_pretty_print(new_bd)

new_bd.save('new_model_data_aug/sunglasses_bd_net.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)
/usr/local/lib/python3.7/dist-packages/keras/layers/wrappers.py:64: CustomMaskWarning: Custom mask layers require a config and mus

Epoch 1/5
1300/1300 [==============================] - 141s 105ms/step - loss: nan - accuracy: 8.6603e-04 - val_loss: nan - val_accuracy: 4.3299e-04
Epoch 2/5
1300/1300 [==============================] - 134s 103ms/step - loss: nan - accuracy: 8.1792e-04 - val_loss: nan - val_accuracy: 4.3299e-04
Epoch 3/5
1300/1300 [==============================] - 136s 104ms/step - loss: nan - accuracy: 8.1792e-04 - val_loss: nan - val_accuracy: 4.3299e-04
Epoch 4/5
1300/1300 [==============================] - 134s 103ms/step - loss: nan - accuracy: 8.1792e-04 - val_loss: nan - val_accuracy: 4.3299e-04
Epoch 5/5
1300/1300 [==============================] - 134s 103ms/step - loss: nan - accuracy: 8.1792e-04 - val_loss: nan - val_accuracy: 4.3299e-04


## Note
Due to Colab RAM limit, we can not keep 4 models in the RAM. So we reuse the variable name `bd` and `new_bd`.



In [16]:
bd = keras.models.load_model(bd2_path)
bd.load_weights(bd2_weights)

new_bd = Prune(bd1, clean_validation_x, clean_validation_y)
new_ac['anonymous_1_bd_net'] = single_pretty_print(new_bd)
new_bd.save('new_model_data_aug/anonymous_1_bd_net.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:218: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:225: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  aggregation=tf.VariableAggregation.MEAN)
/usr/local/lib/python3.7/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:238: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=False)
/usr/local/lib/python3.7/dist-packages/keras/layers/wrappers.py:64: CustomMaskWarning: Custom mask layers require a config and mus

Epoch 1/5
1300/1300 [==============================] - 142s 106ms/step - loss: nan - accuracy: 0.0011 - val_loss: nan - val_accuracy: 4.3299e-04
Epoch 2/5
1300/1300 [==============================] - 141s 109ms/step - loss: nan - accuracy: 8.1792e-04 - val_loss: nan - val_accuracy: 4.3299e-04
Epoch 3/5
1300/1300 [==============================] - 135s 104ms/step - loss: nan - accuracy: 8.1792e-04 - val_loss: nan - val_accuracy: 4.3299e-04
Epoch 4/5
1300/1300 [==============================] - 136s 105ms/step - loss: nan - accuracy: 8.1792e-04 - val_loss: nan - val_accuracy: 4.3299e-04
Epoch 5/5
1300/1300 [==============================] - 136s 105ms/step - loss: nan - accuracy: 8.1792e-04 - val_loss: nan - val_accuracy: 4.3299e-04


In [ ]:
bd = keras.models.load_model(bd3_path)
bd.load_weights(bd3_weights)

new_bd = Prune(bd, clean_validation_x, clean_validation_y)
new_ac['anonymous_2_bd_net'] = single_pretty_print(new_bd)
new_bd.save('new_model_data_aug/anonymous_2_bd_net.h5')

In [ ]:
bd = keras.models.load_model(bd4_path)
bd.load_weights(bd4_weights)

new_bd = Prune(bd, clean_validation_x, clean_validation_y)
new_ac['multi_trigger_multi_target_bd_net'] = single_pretty_print(new_bd)
new_bd.save('new_model_data_aug/multi_trigger_multi_target_bd_net.h5')

In [17]:
print('Result for data augment + prune')

for k,v in old_ac.items():
    vn = new_ac[k]
    if k == 'multi_trigger_multi_target_bd_net':
        print(f'Before training {k} ......')
        print(f'Eyebrows AC = {v[0]:.2f}%')
        print(f'Lipstick AC = {v[1]:.2f}%')
        print(f'Sunglass AC = {v[2]:.2f}%')

        print(f'After training {k} ......')
        print(f'Eyebrows AC = {vn[0]:.2f}%')
        print(f'Lipstick AC = {vn[1]:.2f}%')
        print(f'Sunglass AC = {vn[2]:.2f}%')
    else:
        print(f'Before training {k} ......')
        print(f'Clean test AC = {v[0]:.2f}%')
        print(f'Sunglass AC = {v[1]:.2f}%')
        print(f'Anonymous 1 AC = {v[2]:.2f}%')

        print(f'After training {k} ......')
        print(f'Clean test AC = {vn[0]:.2f}%')
        print(f'Sunglass AC = {vn[1]:.2f}%')
        print(f'Anonymous 1 AC = {vn[2]:.2f}%')

Result for data augment + prune
Before training sunglasses_bd_net ......
Clean test AC = 42.36%
Sunglass AC = 99.95%
Anonymous 1 AC = 1.30%
After training sunglasses_bd_net ......
Clean test AC = 0.08%
Sunglass AC = 100.00%
Anonymous 1 AC = 100.00%
Before training anonymous_1_bd_net ......
Clean test AC = 56.23%
Sunglass AC = 0.00%
Anonymous 1 AC = 91.48%
After training anonymous_1_bd_net ......
Clean test AC = 0.08%
Sunglass AC = 100.00%
Anonymous 1 AC = 100.00%
